In [1]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from mnist import load_dataset
X_train,y_train,X_val,y_val,X_test,y_test = load_dataset()

print (X_train.shape,y_train.shape)

((50000, 1, 28, 28), (50000,))


In [3]:
input_X = T.tensor4("X")
input_shape = [None,1,28,28]
target_y = T.vector("target Y integer",dtype='int32')

#### То же самое сделаем для полносвязной сети

In [4]:
input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

dense_1 = lasagne.layers.DenseLayer(input_layer, num_units=128, 
                                    nonlinearity=lasagne.nonlinearities.rectify,
                                   name='dense_1', b=None)

dense_2 = lasagne.layers.DenseLayer(dense_1, num_units=256, 
                                    nonlinearity=lasagne.nonlinearities.rectify,
                                   name='dense_2', b=None)

dense_3 = lasagne.layers.DenseLayer(dense_2, num_units=64, 
                                    nonlinearity=lasagne.nonlinearities.rectify,
                                   name='dense_2', b=None)

dense_output = lasagne.layers.DenseLayer(dense_3, num_units = 10,
                                        nonlinearity = lasagne.nonlinearities.softmax,
                                        name='output', b=None)

In [5]:
y_predicted = lasagne.layers.get_output(dense_output)

In [6]:
all_weights = lasagne.layers.get_all_params(dense_output)
print (all_weights)

[dense_1.W, dense_2.W, dense_2.W, output.W]


In [7]:
# добавим регуляризацию
reg_1 = lasagne.regularization.regularize_network_params(dense_1, lasagne.regularization.l2)

loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean() + reg_1
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()
updates = lasagne.updates.adadelta(loss, all_weights,learning_rate=0.01)

In [8]:
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates)
accuracy_fun = theano.function([input_X,target_y],accuracy)

In [9]:
from random import shuffle
import math

def iterate_minibatches(X, y, batchsize):
    
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    for start_idx in range(0, len(X) - batchsize + 1, batchsize):
        excerpt = indices[start_idx:start_idx + batchsize]
        yield X[excerpt], y[excerpt]

In [10]:
import time
from tqdm import tqdm

num_epochs = 150 #количество проходов по данным

batch_size = 50 #размер мини-батча

train_accuracy = []
val_accuracy = []

for epoch in tqdm(range(num_epochs)):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batch_size):
        
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))
    
    train_accuracy.append(train_acc / train_batches * 100)
    val_accuracy.append(val_acc / val_batches * 100)

  1%|          | 1/150 [00:08<20:41,  8.33s/it]

Epoch 1 of 150 took 8.332s
  training loss (in-iteration):		91.882278
  train accuracy:		48.01 %
  validation accuracy:		68.92 %
Epoch 2 of 150 took 7.957s

  1%|▏         | 2/150 [00:16<20:16,  8.22s/it]


  training loss (in-iteration):		4.489486
  train accuracy:		72.38 %
  validation accuracy:		80.77 %
Epoch 3 of 150 took 8.024s

  2%|▏         | 3/150 [00:24<19:59,  8.16s/it]

...


  training loss (in-iteration):		0.143724
  train accuracy:		97.49 %
  validation accuracy:		97.00 %
Epoch 149 of 150 took 10.956s

 99%|█████████▉| 149/150 [24:59<00:10, 10.58s/it]


  training loss (in-iteration):		0.142447
  train accuracy:		97.48 %
  validation accuracy:		97.14 %
Epoch 150 of 150 took 11.391s

100%|██████████| 150/150 [25:10<00:00, 10.82s/it]


  training loss (in-iteration):		0.143370
  train accuracy:		97.46 %
  validation accuracy:		97.19 %


In [11]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 99:
    print ("Achievement unlocked: колдун 80 уровня")
else:
    print ("Нужно больше магии!")

Final results:
  test accuracy:		97.00 %
Нужно больше магии!


In [72]:
# теперь попробуем вывести все параметры нашей сети
print("params: ", lasagne.layers.get_all_param_values(dense_output))

('params: ', [array([[  1.19563886e-321,   1.21540149e-321,  -1.19563886e-321, ...,
         -1.19563886e-321,  -1.20552018e-321,   1.18575755e-321],
       [ -1.22528280e-321,  -1.22528280e-321,  -1.21540149e-321, ...,
         -1.22528280e-321,   1.22528280e-321,  -1.18575755e-321],
       [ -1.18575755e-321,  -1.21540149e-321,   1.22528280e-321, ...,
         -1.22528280e-321,   1.21540149e-321,   1.22528280e-321],
       ..., 
       [  1.19563886e-321,   1.21540149e-321,   1.22528280e-321, ...,
          1.20552018e-321,  -1.21540149e-321,   1.22528280e-321],
       [  1.22528280e-321,   1.21540149e-321,   1.22528280e-321, ...,
         -1.19563886e-321,   1.21540149e-321,  -1.21540149e-321],
       [  1.18575755e-321,  -1.21540149e-321,   1.20552018e-321, ...,
         -1.22528280e-321,   1.21540149e-321,   1.21540149e-321]]), array([-0.0068236 ,  0.0094755 ,  0.04961613, -0.00534112,  0.01234983,
        0.04865208,  0.02783767,  0.07810512,  0.00516018, -0.00504561,
        0.0

In [12]:
#сохраняем всё в файлик
np.savez('dense_weights.npz', *lasagne.layers.get_all_param_values(dense_output))

In [14]:
#так можно установить веса
with np.load('dense_weights.npz') as f:
    param_values = [f['arr_%d' % i] for i in range(len(f.files))]
    lasagne.layers.set_all_param_values(dense_output, param_values)